In [1]:
import os
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT","").strip(),
    api_key        = os.getenv("AZURE_OPENAI_API_KEY"),
    api_version    = os.getenv("OPENAI_API_VERSION")
)

deployment_name    = os.getenv('DEPLOYMENT_NAME')

In [2]:
system_msg = """Postgres SQL tables, with their properties:"""
user_msg = """#Employee(id, name, department_id)
# Department(id, name, address)
# Salary_Payments(id, employee_id, amount, date)
#
## A query to list the names of the departments which employed more than 10 employees in the last 3 months
query: SELECT"""
# prompt="""### Postgres SQL tables, with their properties:\n#\n# Employee(id, name, department_id)\n# Department(id, name, address)\n# Salary_Payments(id, employee_id, amount, date)\n#\n###
# 최근 3 개월간 직원을 10명 이상 뽑은 부서의 이름을 수집하는 쿼리  \n\ query: SELECT"""

"""한글 결과
Department.name FROM Department
INNER JOIN Employee ON Department.id = Employee.department_id
INNER JOIN Salary_Payments ON Employee.id = Salary_Payments.employee_id
WHERE Salary_Payments.date >= NOW() - INTERVAL '3 months'
GROUP BY Department.id, Department.name
HAVING COUNT(DISTINCT Employee.id) >= 10
"""

response = client.chat.completions.create(
    model=deployment_name,
    messages=[
        {"role": "system", "content": system_msg},
        {"role": "user", "content": user_msg},
    ],
  temperature=0,
  max_tokens=150
)

print(response.choices[0].message.content)

Department.name
FROM Department
WHERE Department.id IN (
    SELECT department_id
    FROM Employee
    WHERE Employee.id IN (
        SELECT employee_id
        FROM Salary_Payments
        WHERE date >= current_date - interval '3 months'
    )
    GROUP BY department_id
    HAVING COUNT(*) > 10
)


In [3]:
system_msg = """Can you explain what does this code do?"""
user_msg = """Code: SELECT d.name FROM Department d JOIN Employee e ON d.id = e.department_id WHERE e.id IN (SELECT employee_id FROM Salary_Payments WHERE date > now() - interval '3 months') GROUP BY d.name HAVING COUNT(*) > 10
   Answer: # """

# system_msg = """"이 코드가 무엇을 수행하는지 설명해줘?"""
# user_msg = """Code:
# SELECT d.name FROM Department d JOIN Employee e ON d.id = e.department_id WHERE e.id IN (SELECT employee_id FROM Salary_Payments WHERE date > now() - interval '3 months') GROUP BY d.name HAVING COUNT(*) > 10
# Answer: # """
   
response = client.chat.completions.create(
    model=deployment_name,
    messages=[
        {"role": "system", "content": system_msg},
        {"role": "user", "content": user_msg},
    ],
   max_tokens=250)

print(response.choices[0].message.content)

This code is a SQL query that retrieves the names of departments that have more than 10 employees who have received a salary payment in the last 3 months. 

The query starts by selecting the department's name from the "Department" table. It then joins the "Employee" table using the department id, and specifies a condition that the employee id must be included in the subquery. 

The subquery selects the employee id from the "Salary_Payments" table where the date is greater than the current date minus 3 months. This ensures that only employees who have received a salary payment in the last 3 months are included. 

The query then groups the results by department name and applies a "HAVING" clause to filter out any departments that have fewer than or equal to 10 employees. The result is a list of department names that have more than 10 employees who received a salary payment in the last 3 months.
